In [155]:
import pandas as pd

data = pd.read_excel('./data/MPVDatasetDownload.xlsx')
data.head()

,Victim's name,Victim's age,Victim's gender,Victim's race,URL of image of victim,Date of Incident (month/day/year),Street Address of Incident,City,State,Zipcode,...,Unarmed/Did Not Have an Actual Weapon,Alleged Weapon (Source: WaPo and Review of Cases Not Included in WaPo Database),Alleged Threat Level (Source: WaPo),Fleeing (Source: WaPo),Body Camera (Source: WaPo),WaPo ID (If included in WaPo database),Off-Duty Killing?,Geography (via Trulia methodology based on zipcode population density: http://jedkolko.com/wp-content/uploads/2015/05/full-ZCTA-urban-suburban-rural-classification.xlsx ),MPV ID,Fatal Encounters ID
0,Pedro Martinez,66,Male,Hispanic,NaN,2020-11-18,NaN,Tyler,TX,NaN,...,Allegedly Armed,unknown weapon,other,not fleeing,no,6321.0,NaN,NaN,NaN,NaN
1,James Horton,42,Male,NaN,NaN,2020-11-17,NaN,Hart County,KY,NaN,...,Allegedly Armed,gun,attack,other,no,6322.0,NaN,NaN,NaN,NaN
2,Dustin James Acosta,28,Male,White,NaN,2020-11-17,NaN,St. Augustine,FL,NaN,...,Allegedly Armed,gun,attack,NaN,No,6320.0,NaN,NaN,NaN,NaN
3,Javon Brice,39,Male,NaN,NaN,2020-11-17,NaN,La Vergne,TN,NaN,...,Allegedly Armed,gun,attack,car,no,6319.0,NaN,NaN,NaN,NaN
4,Joshua D. Evans,32,Male,NaN,NaN,2020-11-16,NaN,Batesville,IN,NaN,...,Allegedly Armed,gun,attack,not fleeing,no,6314.0,NaN,NaN,NaN,NaN


In [156]:
len(data)

8629

In [157]:
data.columns

Index(['Victim's name', 'Victim's age', 'Victim's gender', 'Victim's race',
       'URL of image of victim', 'Date of Incident (month/day/year)',
       'Street Address of Incident', 'City', 'State', 'Zipcode', 'County',
       'Agency responsible for death', 'ORI Agency Identifier (if available)',
       'Cause of death',
       'A brief description of the circumstances surrounding the death',
       'Official disposition of death (justified or other)',
       'Criminal Charges?',
       'Link to news article or photo of official document',
       'Symptoms of mental illness?', 'Unarmed/Did Not Have an Actual Weapon',
       'Alleged Weapon (Source: WaPo and Review of Cases Not Included in WaPo Database)',
       'Alleged Threat Level (Source: WaPo)', 'Fleeing (Source: WaPo)',
       'Body Camera (Source: WaPo)', 'WaPo ID (If included in WaPo database)',
       'Off-Duty Killing?',
       'Geography (via Trulia methodology based on zipcode population density: http://jedkolko.com/wp-co

In [158]:
data.drop(["Victim's name",
           "URL of image of victim", "Street Address of Incident", "Zipcode", "County", "ORI Agency Identifier (if available)", 
           "A brief description of the circumstances surrounding the death",
           "Official disposition of death (justified or other)",
           "Link to news article or photo of official document", "Symptoms of mental illness?",
           "Alleged Threat Level (Source: WaPo)", #3968 are attack, the rest are undetermined/other
           "Body Camera (Source: WaPo)", "WaPo ID (If included in WaPo database)",
           "Off-Duty Killing?", #239/8629 are not NaN
           "Geography (via Trulia methodology based on zipcode population density: http://jedkolko.com/wp-content/uploads/2015/05/full-ZCTA-urban-suburban-rural-classification.xlsx )",
           "MPV ID", "Fatal Encounters ID"], 
          inplace=True, axis=1)

In [159]:
data.head()

,Victim's age,Victim's gender,Victim's race,Date of Incident (month/day/year),City,State,Agency responsible for death,Cause of death,Criminal Charges?,Unarmed/Did Not Have an Actual Weapon,Alleged Weapon (Source: WaPo and Review of Cases Not Included in WaPo Database),Fleeing (Source: WaPo)
0,66,Male,Hispanic,2020-11-18,Tyler,TX,Tyler Junior College Police Department,Gunshot,No known charges,Allegedly Armed,unknown weapon,not fleeing
1,42,Male,NaN,2020-11-17,Hart County,KY,"Breckinridge County Sheriff's Office, Kentucky...",Gunshot,No known charges,Allegedly Armed,gun,other
2,28,Male,White,2020-11-17,St. Augustine,FL,"St. Augustine Police Department, St. Johns Cou...",Gunshot,No known charges,Allegedly Armed,gun,NaN
3,39,Male,NaN,2020-11-17,La Vergne,TN,La Vergne Police Department,Gunshot,No known charges,Allegedly Armed,gun,car
4,32,Male,NaN,2020-11-16,Batesville,IN,"Batesville Police Department, Greensburg Polic...",Gunshot,No known charges,Allegedly Armed,gun,not fleeing


In [160]:
data['Cause of death'].unique()

array(['Gunshot', 'Gunshot, Taser', 'Physical Restraint', 'Tasered',
       'Vehicle', 'Chemical agent/Pepper spray', 'Beaten', 'Taser',
       'Beaten/Bludgeoned with instrument', 'Other',
       'Gunshot, Beanbag Gun', 'Gunshot, Police Dog',
       'Gunshot, Bean Bag Gun', 'Gunshot, Pepper Spray', 'Bean bag',
       'Gunshot, Taser, Baton', 'Gunshot, Taser, Beanbag Shotgun',
       'Pepper Spray', 'Asphyxiated',
       'Gunshot, Unspecified Less Lethal Weapon', 'Physical restraint',
       'Taser, Baton', 'Bomb', 'Taser, Physical Restraint',
       'Baton, Pepper Spray, Physical Restraint', 'Gunshot, Vehicle',
       'Gunshot, Stabbed', 'Gunshot, Taser, Pepper spray',
       'Taser, Pepper Spray, Beaten', 'Taser, Beaten',
       'Taser, Pepper spray, beaten'], dtype=object)

In [161]:
data['Alleged Weapon (Source: WaPo and Review of Cases Not Included in WaPo Database)'].unique()

array(['unknown weapon', 'gun', 'undetermined', 'vehicle', 'knife',
       'gun, vehicle', 'curtain rod', 'sharp object', 'machete',
       'no object', 'hatchet', 'shovel', 'Airsoft pistol', 'toy weapon',
       'pepper spray', 'metal pipe', 'bottle', 'gun and knife', 'sword',
       'baseball bat', 'Taser', 'pellet gun', 'vehicle and gun', 'hammer',
       'rock', 'samurai sword', 'gun and car', 'screwdriver',
       'windshield wiper', 'gun and vehicle', 'pick-axe', 'tire iron',
       'BB gun', 'scissors', 'meat cleaver', 'chair', 'chain',
       'car, knife and mace', 'vehicle and machete', 'box cutter', 'ax',
       'crowbar', 'baseball bat and knife', 'straight edge razor',
       'baton', 'air pistol', 'wasp spray', 'BB gun and vehicle',
       'piece of wood', 'garden tool', 'barstool', 'walking stick',
       'wrench', 'beer bottle', 'flag pole', 'lawn mower blade',
       'crossbow', 'metal object', 'bow and arrow', 'lamp', 'glass shard',
       'incendiary device', 'pipe', 

In [162]:
data['Criminal Charges?'].unique()

array(['No known charges', 'Charged with a crime',
       'Charged, Convicted, Sentenced to 7 years in prison',
       'Charged, Acquitted',
       'Charged, Convicted, Sentenced to 1 year in prison and 4 years probation',
       'Charged, Charges Dropped',
       'Charged, Convicted, Sentenced to 10 years in prison',
       'Charged, Convicted', 'Charged with a crime, Acquitted',
       'Charged, Plead No Contest, Sentenced to 10 years probation',
       'Charged, Convicted, Sentenced to 6 years in prison',
       'Charged, Convicted of reckless homicide, sentenced to 1 year in prison',
       'Charged, Convicted, Sentenced to 14 years in prison',
       'Charged, Convicted, Sentenced to 5 years in prison',
       'Charged, Convicted, Sentenced to 40 years in prison',
       'Charged, Convicted, Sentenced to 25 years in prison',
       'Charged, Convicted, Sentenced to 3 months in jail',
       'Charged, Mistrial',
       'Charged, Convicted, Sentenced to 30 years in prison',
       '

In [163]:
data.loc[data['Criminal Charges?'] != "No known charges", 'Criminal Charges?'] = "Charged"
data['Criminal Charges?'].unique()

array(['No known charges', 'Charged'], dtype=object)

In [164]:
data['Fleeing (Source: WaPo)'].unique()

array(['not fleeing', 'other', nan, 'car', 'foot', 'Not fleeing', 'Foot',
       'Car', 'Not Fleeing', 'Other'], dtype=object)

In [165]:
data['Fleeing (Source: WaPo)']=data['Fleeing (Source: WaPo)'].str.lower()
data['Fleeing (Source: WaPo)'].unique()

array(['not fleeing', 'other', nan, 'car', 'foot'], dtype=object)

In [166]:
data.groupby('Fleeing (Source: WaPo)').count()

,Victim's age,Victim's gender,Victim's race,Date of Incident (month/day/year),City,State,Agency responsible for death,Cause of death,Criminal Charges?,Unarmed/Did Not Have an Actual Weapon,Alleged Weapon (Source: WaPo and Review of Cases Not Included in WaPo Database)
Fleeing (Source: WaPo),,,,,,,,,,,
car,946,946,947,949,948,949,947,949,949,949,949
foot,741,742,742,742,742,742,742,742,742,742,742
not fleeing,3793,3803,3798,3804,3802,3804,3803,3804,3804,3804,3804
other,178,179,178,179,178,179,179,179,179,179,179


In [167]:
data = data.rename(columns={"Victim's age": 'victim_age',
                            "Victim's gender": 'victim_gender',
                            "Victim's race": 'victim_race',
                            "Date of Incident (month/day/year)": 'date',
                            #"Agency responsible for death": 'agency_responsible',
                            "Cause of death": 'cause_of_death',
                            "Criminal Charges?": 'criminal_charges',
                            "Unarmed/Did Not Have an Actual Weapon": 'victim_unarmed',
                            "Alleged Weapon (Source: WaPo and Review of Cases Not Included in WaPo Database)": 'alleged_weapon',
                            "Alleged Threat Level (Source: WaPo)": 'alleged_threat_level',
                            "Fleeing (Source: WaPo)": 'victim_fleeing'})

In [168]:
data.head()

,victim_age,victim_gender,victim_race,date,City,State,Agency responsible for death,cause_of_death,criminal_charges,victim_unarmed,alleged_weapon,victim_fleeing
0,66,Male,Hispanic,2020-11-18,Tyler,TX,Tyler Junior College Police Department,Gunshot,No known charges,Allegedly Armed,unknown weapon,not fleeing
1,42,Male,NaN,2020-11-17,Hart County,KY,"Breckinridge County Sheriff's Office, Kentucky...",Gunshot,No known charges,Allegedly Armed,gun,other
2,28,Male,White,2020-11-17,St. Augustine,FL,"St. Augustine Police Department, St. Johns Cou...",Gunshot,No known charges,Allegedly Armed,gun,NaN
3,39,Male,NaN,2020-11-17,La Vergne,TN,La Vergne Police Department,Gunshot,No known charges,Allegedly Armed,gun,car
4,32,Male,NaN,2020-11-16,Batesville,IN,"Batesville Police Department, Greensburg Polic...",Gunshot,No known charges,Allegedly Armed,gun,not fleeing


I think we still can drop `agency_responsible`
***

In [169]:
url = r'https://giffords.org/lawcenter/resources/scorecard/#rankings'
tables = pd.read_html(url) # Returns list of all tables on page
table = tables[0] # Select table of interest

In [172]:
table.head()

,state_gunlaw_strength,State,state_gunlaw_grade,state_gun_death_rate,state_gun_death_rate_per_100K
0,38,Alabama,F,2,21.70
1,42,Alaska,F,7,20.74
2,45,Arizona,F,18,15.29
3,40,Arkansas,F,8,18.96
4,1,California,A,44,7.45


In [171]:
table = table.rename(columns={"Gun LawStrength(Ranked)": 'state_gunlaw_strength',
                            "2019Grade": 'state_gunlaw_grade',
                            "Gun DeathRate(Ranked)": 'state_gun_death_rate',
                            "Gun DeathRate(Per 100K)": 'state_gun_death_rate_per_100K'})
table.head()

,state_gunlaw_strength,State,state_gunlaw_grade,state_gun_death_rate,state_gun_death_rate_per_100K
0,38,Alabama,F,2,21.70
1,42,Alaska,F,7,20.74
2,45,Arizona,F,18,15.29
3,40,Arkansas,F,8,18.96
4,1,California,A,44,7.45


In [178]:
#We probably should put this in a file
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

In [176]:
table = table.replace({"State": us_state_abbrev})
table.head()

,state_gunlaw_strength,State,state_gunlaw_grade,state_gun_death_rate,state_gun_death_rate_per_100K
0,38,AL,F,2,21.70
1,42,AK,F,7,20.74
2,45,AZ,F,18,15.29
3,40,AR,F,8,18.96
4,1,CA,A,44,7.45


***

Adding gun law information as a feature to Police Violence dataset

In [179]:
data = pd.merge(data, table, on='State')

In [180]:
data.head()

,victim_age,victim_gender,victim_race,date,City,State,Agency responsible for death,cause_of_death,criminal_charges,victim_unarmed,alleged_weapon,victim_fleeing,state_gunlaw_strength,state_gunlaw_grade,state_gun_death_rate,state_gun_death_rate_per_100K
0,66,Male,Hispanic,2020-11-18,Tyler,TX,Tyler Junior College Police Department,Gunshot,No known charges,Allegedly Armed,unknown weapon,not fleeing,34,F,30,12.22
1,50,Female,NaN,2020-11-15,Lubbock,TX,Lubbock Police Department,Gunshot,No known charges,Allegedly Armed,unknown weapon,not fleeing,34,F,30,12.22
2,66,Male,Unknown race,2020-11-14,Corpus Christi,TX,Corpus Christi Police Department,Gunshot,No known charges,Allegedly Armed,gun,NaN,34,F,30,12.22
3,29,Male,Unknown race,2020-11-10,Nederland,TX,Nederland Police Department,Gunshot,No known charges,Allegedly Armed,gun,foot,34,F,30,12.22
4,25,Male,Black,2020-11-05,Dallas,TX,Dallas Police Department,Gunshot,No known charges,Allegedly Armed,gun,NaN,34,F,30,12.22
